In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import neighborhoods as hoods

In [39]:
df = pd.read_csv('eviction_notices.csv', parse_dates=['File Date'], encoding = "ISO-8859-1")

/Users/jakram/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
df

,Eviction ID,Address,City,State,Eviction Notice Source Zipcode,File Date,Non Payment,Breach,Nuisance,Illegal Use,...,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Constraints Date,Supervisor District,Neighborhoods - Analysis Boundaries,Location
0,M161957,300 Block Of Park Street,San Francisco,CA,94110,2016-06-06,False,False,False,False,...,False,False,False,False,False,False,NaN,9.0,Bernal Heights,"(37.7364374645373, -122.417989910582)"
1,M162256,200 Block Of Seneca Avenue,San Francisco,CA,94112,2016-06-29,False,False,False,False,...,False,False,False,False,False,False,8/25/2021,11.0,Outer Mission,"(37.7205429316262, -122.443264852669)"
2,M162135,200 Block Of Dolores Street,San Francisco,CA,94103,2016-06-28,False,True,False,False,...,False,False,False,False,False,False,NaN,8.0,Castro/Upper Market,"(37.7652067507312, -122.426591617441)"
3,M161901,1200 Block Of 9th Avenue,San Francisco,CA,94122,2016-06-02,False,False,False,False,...,False,False,False,False,False,False,NaN,5.0,Inner Sunset,"(37.764977785911, -122.4664456379)"
4,M162428,1400 Block Of Larkin Street,San Francisco,CA,94109,2016-02-28,False,False,False,False,...,False,False,False,False,False,False,NaN,3.0,Nob Hill,"(37.7912373166567, -122.419197310472)"
5,M161943,300 Block Of Wilde Avenue,San Francisco,CA,94134,2016-06-06,False,False,True,False,...,False,False,False,False,False,False,NaN,10.0,Visitacion Valley,"(37.7167969475402, -122.403664398767)"
6,M161986,0 Block Of Fair Avenue,San Francisco,CA,94110,2016-06-13,False,True,False,False,...,False,False,False,False,False,False,NaN,9.0,Bernal Heights,"(37.7453025419492, -122.419008184627)"
7,M161976,1300 Block Of Market Street,San Francisco,CA,94102,2016-06-13,True,False,False,False,...,False,False,False,False,False,False,NaN,6.0,Tenderloin,"(37.777083463941, -122.416979326588)"
8,M162257,300 Block Of Lombard Street,San Francisco,CA,94133,2016-06-30,False,False,False,False,...,False,False,False,False,False,False,8/27/2021,3.0,North Beach,"(37.8035109665788, -122.407095028398)"
9,M161838,2800 Block Of Pierce Street,San Francisco,CA,94123,2016-06-01,False,False,False,False,...,False,False,False,False,False,False,8/1/2021,2.0,Marina,"(37.7961971224445, -122.438753767562)"


In [41]:
byhood = pd.DataFrame(df.groupby(['Neighborhoods - Analysis Boundaries']).agg({'Eviction ID' : ['count']})).reset_index()
byhood

,Neighborhoods - Analysis Boundaries,Eviction ID
,,count
0,Bayview Hunters Point,995
1,Bernal Heights,1100
2,Castro/Upper Market,1738
3,Chinatown,402
4,Excelsior,1111
5,Financial District/South Beach,238
6,Glen Park,254
7,Golden Gate Park,5
8,Haight Ashbury,1241


In [47]:
aggregation = {
    'Owner Move In' : {
        'total_move_in' : 'sum'
    },
    'Condo Conversion' : {
         'total_condo' : 'sum'
    },
    '' : {
        '' : 'sum'
    }
}
df.groupby(['Neighborhoods - Analysis Boundaries']).agg(aggregation).reset_index()

,Neighborhoods - Analysis Boundaries,Breach,Condo Conversion
,,total_breach,total_condo
0,Bayview Hunters Point,215.0,0.0
1,Bernal Heights,152.0,6.0
2,Castro/Upper Market,321.0,17.0
3,Chinatown,70.0,3.0
4,Excelsior,203.0,0.0
5,Financial District/South Beach,63.0,0.0
6,Glen Park,25.0,3.0
7,Golden Gate Park,1.0,0.0
8,Haight Ashbury,228.0,5.0


In [33]:
df.dtypes

Eviction ID                                    object
Address                                        object
City                                           object
State                                          object
Eviction Notice Source Zipcode                 object
File Date                              datetime64[ns]
Non Payment                                      bool
Breach                                           bool
Nuisance                                         bool
Illegal Use                                      bool
Failure to Sign Renewal                          bool
Access Denial                                    bool
Unapproved Subtenant                             bool
Owner Move In                                    bool
Demolition                                       bool
Capital Improvement                              bool
Substantial Rehab                                bool
Ellis Act WithDrawal                             bool
Condo Conversion            